# Top tracks populars
Los datos que se ocupan en este Notebook son del siguiente link de kaggle : https://www.kaggle.com/datasets/zeesolver/spotfy

## Importar dependencias

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import date

pd.options.display.max_columns = None
# pd.options.display.max_rows = None
# pd.options.display.max_columns = 20 #Default
# pd.options.display.max_rows = 60 # Default

## Funciones

In [2]:
def bar_label_h(df_col_label,df_col_value):
    fig = go.Figure(
        data=go.Bar(
                x=df_col_value, 
            ),
        layout={
            'bargroupgap':0.4, 
            'yaxis':{'visible': False},
            'xaxis':{'visible': False},
            'barcornerradius':5,
        })
    df_col_value = df_col_value.map('{:,.0f}'.format)
    label = df_col_label + ": " + df_col_value

    for idx, name in enumerate(label):
        fig.add_annotation(
            x=0,
            y=idx + 0.45,
            text=name,
            xanchor='left',
            showarrow=False,
            yshift=0
        )
    return fig

def line_interval(x,y,min_x=None,max_x=None):
    r_min = x.min() if min_x==None else date.fromisoformat(min_x)
    r_max = x.max() if max_x==None else date.fromisoformat(max_x)
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=x, 
        y=y,
        text= y,
        textposition='top center',
        textfont=dict(color='#000000'),
        line_color='rgb(0,100,80)',
    ))
    fig.update_layout(
        xaxis = {'range':[r_min,r_max]}
    )
    fig.update_traces(mode='lines+text')
    return fig

def group_agg(df,ls):
    df = df[ls].mean()
    df = pd.DataFrame(data=df).reset_index()
    df.columns = ['type','value']
    return df

## Importar los datos

In [3]:
data = pd.read_csv('Popular_Spotify_Songs.csv',encoding='latin1')

## Checamos el dataframe y alineamos el tipo de dato de cada columna

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

In [10]:
data['mode'].unique()

array(['Major', 'Minor'], dtype=object)

Detectamos que las siguientes columnas deben tener la siguiente estructura

    -streams : float o int
    -in_deezer_playlists : float o int
    -in_shazam_charts : float o int

## Encontrar valores que no pueden cambiar de tipo a los tipos correctos

Primero creamos una funcion para validar / encontrar errores en el cambio de tipo

Despues la ejecutamos con las columnas en los cuales no poemos hacer el cambio de tipo

In [5]:
def error_str_to_int(df_column,flg_print=True):    
    ls_error = []
    for i in df_column.unique():
        try:
            int(i)
        except:
            if len(ls_error) <6:
                if flg_print:
                    print(i)
            ls_error = ls_error + [i]
    if flg_print:
        print(f'EL numero de errores es {len(ls_error)}')
    return ls_error
#_____________________________________________________________
#Errores
dic_error = {}
ls_val = [
    'streams',
    'in_deezer_playlists',
    'in_shazam_charts',
]
for col_name in ls_val:
    print('________________________')
    print(col_name)
    ls_error = error_str_to_int(data[col_name])
    dic_error.update({col_name:ls_error})

________________________
streams
BPM110KeyAModeMajorDanceability53Valence75Energy69Acousticness7Instrumentalness0Liveness17Speechiness3
EL numero de errores es 1
________________________
in_deezer_playlists
2,445
3,394
3,421
4,053
1,056
4,095
EL numero de errores es 76
________________________
in_shazam_charts
1,021
1,281
nan
1,173
1,093
1,133
EL numero de errores es 8


### streams

Observamos que existe un registro en el cual streams es una cadena de texto la cual no se puede transformar en numero float o int, streams es una columna importante para el analisis por lo que vamos a descartar este registro del analisis

In [6]:
data = data[~data['streams'].isin(dic_error['streams'])]
data['streams'] = data['streams'].fillna(0).astype(float)

### in_shazam_charts y in_deezer_playlists
Observamos que el tema de esta columna es que los numero estan tienen formato de comas para separar cada 3 cifras (Miles y millones)

En el caso de **in_shazam_charts** los valores nulos los sustituimos por cero

In [7]:
data['in_shazam_charts'] = data['in_shazam_charts'].str.replace(',','').fillna(0).astype(int)
data['in_deezer_playlists'] = data['in_deezer_playlists'].str.replace(',','').astype(int)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 952 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_name            952 non-null    object 
 1   artist(s)_name        952 non-null    object 
 2   artist_count          952 non-null    int64  
 3   released_year         952 non-null    int64  
 4   released_month        952 non-null    int64  
 5   released_day          952 non-null    int64  
 6   in_spotify_playlists  952 non-null    int64  
 7   in_spotify_charts     952 non-null    int64  
 8   streams               952 non-null    float64
 9   in_apple_playlists    952 non-null    int64  
 10  in_apple_charts       952 non-null    int64  
 11  in_deezer_playlists   952 non-null    int32  
 12  in_deezer_charts      952 non-null    int64  
 13  in_shazam_charts      952 non-null    int32  
 14  bpm                   952 non-null    int64  
 15  key                   857 no

Así dejamos alineado el dataframe en cuestion de los tipos

## Canciones con el mismo nombre
Observamos que existen tracks con el mismo nombre pero a nivel regitro hay diferencias así que vamos a suponer que son tracks distintos

In [9]:
data['FLG_DUPLICATE'] = data.groupby('track_name',as_index=False)['track_name'].transform('count')
data_dup = data[data['FLG_DUPLICATE']>1].sort_values('track_name',ascending=False)
l_df_dup = len(data_dup)
l_df_dup_unique = len(data_dup.drop_duplicates())
mensaje = f'''
El tamaño de la tabla con nombres tracks duplicados es {l_df_dup}, 
quitando los duplicados a nivel registro son {l_df_dup_unique}
'''
print(mensaje)


El tamaño de la tabla con nombres tracks duplicados es 20, 
quitando los duplicados a nivel registro son 20



## Top de canciones
El top de canciones sera determinado por numero de streams

In [10]:
data = data.sort_values('streams',ascending=False).reset_index(drop=True)
data = data.reset_index()
#Crear el top de canciones
data = data.rename(columns={'index':'top'})
data['top'] = data['top'] + 1

In [11]:
data_top = data[data['top']<=10][['streams','track_name']]
data_top = data_top.sort_values('streams')

In [31]:
bar_label_h(data_top['track_name'],data_top['streams'])

## Artistas con mayor numero de canciones
Primero observamos que hay canciones con más de un artista vease en la columna **artist_count**

In [13]:
data[data['artist_count'] >1][['top','track_name','artist(s)_name']].head()

,top,track_name,artist(s)_name
4,5,Sunflower - Spider-Man: Into the Spider-Verse,"Post Malone, Swae Lee"
5,6,One Dance,"Drake, WizKid, Kyla"
6,7,STAY (with Justin Bieber),"Justin Bieber, The Kid Laroi"
8,9,Closer,"The Chainsmokers, Halsey"
9,10,Starboy,"The Weeknd, Daft Punk"


### Creación tabla secundaria con un registro por artista

In [14]:
data_artist = data[['top', 'artist(s)_name']]
data_artist['artist(s)_name'] = data_artist['artist(s)_name'].apply(lambda x : x.split(', '))
data_artist = (
    data_artist[['top', 'artist(s)_name']]
    .set_index(['top'])['artist(s)_name']
    .apply(pd.Series)
    .stack()
    .reset_index(level=1, drop=True)
    .reset_index()
    .rename(columns={0:'artist(s)_name'})
)

C:\Users\garni\AppData\Local\Temp\ipykernel_22352\1338897866.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
top_artist_clean = (
    data_artist
        .groupby('artist(s)_name',as_index=False)['top'].count()
        .sort_values('top',ascending=False)
        .reset_index(drop=True)
        .head(10)
)
top_artist_clean.columns = ['artist_clean','N Popular tracks clean']
top_artis = (
    data
        .groupby('artist(s)_name',as_index=False)['top'].count()
        .sort_values('top',ascending=False)
        .reset_index(drop=True)
        .head(10)
)
top_artis.columns = ['artist','N Popular tracks']
#Comparativa
top_comparative = top_artis.merge(top_artist_clean,left_index=True, right_index=True)

In [16]:
top_artist_clean = top_artist_clean.sort_values('N Popular tracks clean')

In [17]:
bar_label_h(top_artist_clean['artist_clean'],top_artist_clean['N Popular tracks clean'])

## Año de realización

In [18]:
data['released_period'] = data['released_year'].astype(str) + '/' + data['released_month'].astype(str) + '/01'
data['released_period'] = pd.to_datetime(data['released_period'],yearfirst=True)
data_period =data.groupby(['released_period'],as_index=False)['top'].count()

In [19]:
data_y = data.groupby(['released_year'],as_index=False)['top'].count()

In [20]:
data_y['cumsum'] = (data_y['top'].cumsum() / data_y['top'].sum())*100

In [21]:
data_y

,released_year,top,cumsum
0,1930,1,0.105042
1,1942,1,0.210084
2,1946,1,0.315126
3,1950,1,0.420168
4,1952,1,0.525210
5,1957,2,0.735294
6,1958,3,1.050420
7,1959,2,1.260504
8,1963,3,1.575630
9,1968,1,1.680672


## Grafico de lineas
El 77% de los datos se encuentran a partir del 2020 por lo tanto el rango lo vemos a partir de 2020

In [22]:
line_interval(data_period['released_period'],data_period['top'],'2020-01-01')

# Comparación
spoty
apple
deezer

In [23]:
data
playlist = [
    'in_spotify_playlists',
    'in_apple_playlists',
    'in_deezer_playlists',
    'in_shazam_charts',
]
charts = [
    'in_spotify_charts',
    'in_apple_charts',
    'in_deezer_charts',
]
print('playlist')
print('_______________________________')
print(data[playlist].mean())
print('__________________________________________________')
print('charts')
print('_______________________________')
print(data[charts].mean())

playlist
_______________________________
in_spotify_playlists    5202.565126
in_apple_playlists        67.866597
in_deezer_playlists      385.535714
in_shazam_charts          56.907563
dtype: float64
__________________________________________________
charts
_______________________________
in_spotify_charts    12.022059
in_apple_charts      51.963235
in_deezer_charts      2.669118
dtype: float64


In [24]:
data_playlist = group_agg(data,playlist)
data_playlist['type'] = data_playlist['type'].apply(lambda x : x.split('_')[1])
data_playlist = data_playlist.sort_values('value')

Observamos que spotify es donde los tracks mas escuchados tienen mayor promedio de playlis

In [25]:
bar_label_h(data_playlist['type'],data_playlist['value'])

In [26]:
data_charts = group_agg(data,charts)
data_charts['type'] = data_charts['type'].apply(lambda x : x.split('_')[1])
data_charts = data_charts.sort_values('value')

Observamos que apple es donde los tracks mas escuchados tienen mayor promedio de charts

In [27]:
bar_label_h(data_charts['type'],data_charts['value'])

# Globalmente como estan ubicados los siguiente indicadores
    danceability_%
    valence_%
    energy_%
    acousticness_%
    instrumentalness_%
    liveness_%
    speechiness_%

In [28]:
ls_indicator = [
    'danceability_%',
    'valence_%',
    'energy_%',
    'acousticness_%',
    'instrumentalness_%',
    'liveness_%',
    'speechiness_%',
]
print('_______________________________')
print('Indicadores')
print('_______________________________')
print(data[ls_indicator].mean())
print('_______________________________')

_______________________________
Indicadores
_______________________________
danceability_%        66.984244
valence_%             51.406513
energy_%              64.274160
acousticness_%        27.078782
instrumentalness_%     1.582983
liveness_%            18.214286
speechiness_%         10.138655
dtype: float64
_______________________________


In [29]:
data_ind = group_agg(data,ls_indicator)
data_ind['type'] = data_ind['type'].apply(lambda x : x.split('_')[0])
data_ind = data_ind.sort_values('value')
ls_ind =data_ind.to_dict(orient='records')

In [30]:
ind = bar_label_h(data_ind['type'],data_ind['value'])
ind.update_layout(
    xaxis = {'range':[0,100],'visible': True}
)